# All imports

In [3]:
import os
import numpy as np
import random
from tqdm import tqdm
import gymnasium as gym
from collections import deque, namedtuple
from collections import defaultdict
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from gymnasium.envs.toy_text.frozen_lake import generate_random_map

# w2v required
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics.pairwise import cosine_similarity

# DQN required
import argparse
import numpy as np
import logging
from matplotlib import animation # will be needed for rendering

# All global variables needed
Since some or all of these variables are needed for each cell below, it's difficult to put these inside the main function.\
Perhaps, using some args method might work?

## Common Functions needed

### Function to generate custom map

In [4]:
# ============================================================================================
# Function to generate custom map
# ============================================================================================
def make_env(env_name, env_dim = 4, seed = 42, stochastic = False):
    env = gym.make(env_name, desc=generate_random_map(size=env_dim, seed=seed), 
                   is_slippery = stochastic, render_mode = 'rgb_array')
    return env

## Global Variables

In [5]:
'''
env_name: str
env_dim: int --> Dimension of the game: 4x4 or 8x8
seed
stochastic = boolean --> Whether we use is_slippery = True or False
 '''
env_name = "FrozenLake-v1"
env_dim = 4
stochastic = False
seed = 42
gamma = 0.99 # discount factor in Q computation
alpha = 0.1 # learning rate in the table
num_episodes_q_table = 100_000
convergence_threshold = 1e-4
epsilon_start = 1
epsilon_decay_q_table = 0.99995
epsilon_end = 0.01
check_env_details = True

# Creating the environment
env = make_env(env_name=env_name, env_dim=env_dim, seed = seed, stochastic=stochastic)
state_dim = env.observation_space.n
action_dim = env.action_space.n
print("State space: ", env.observation_space.n)
print("Action space: ", env.action_space.n)

# state and trajectories related variables
num_episodes_trajectories = 10_000
num_states = state_dim
num_actions = action_dim
max_eps_len = 100
modified = "perfect" # or random - for purely random trajectories or "False" for the combined trajs

# w2v related variables
''' Potential values for embedding dimensions = {4, 8, 12, 16, 20, 32, 64} '''
# w2v hyperparameters
embed_dim = 32
window_size = 2
batch_size = 16
w2v_epochs = 50
w2v_lr = 0.01
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Filename for saving purposes - common part for all files
filename = f"{env_name}_map_size_{env_dim}_stochastic_{stochastic}_seed_{seed}"

# This folder path will be used to store all the saved models and the associated data
runs_folder_path = "mdp/runs_frozen"

# Check if the folder exists
if not os.path.exists(runs_folder_path):
    # Create the folder
    os.makedirs(runs_folder_path)
    print(f"Folder '{runs_folder_path}' created successfully.")
else:
    print(f"Folder '{runs_folder_path}' already exists.")


State space:  16
Action space:  4
Folder 'mdp/runs_frozen' created successfully.


# Tabular Q learning

### Q-Learning Agent class

In [6]:
# ============================================================================================
# Q-Learning Agent class
# ============================================================================================
class QLearningAgent:
    """Q-learning agent."""
    def __init__(self, num_states, num_actions, gamma=0.99, epsilon=0.1, alpha=0.1):
        self.num_states = num_states
        self.num_actions = num_actions
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.q_table = np.zeros((num_states, num_actions))  # Initialize Q-table

    def choose_action(self, state, epsilon):
        """Epsilon-greedy action selection."""
        if random.uniform(0, 1) < epsilon:
            return random.randint(0, self.num_actions - 1)  # Explore
        else:
            return np.argmax(self.q_table[state, :])  # Exploit

    def update_q_value(self, state, action, reward, next_state):
        """Q-learning update rule."""
        best_next_action = np.argmax(self.q_table[next_state, :])  # Greedy action for next state
        td_target = reward + self.gamma * self.q_table[next_state, best_next_action]
        td_error = td_target - self.q_table[state, action]
        self.q_table[state, action] += self.alpha * td_error  # Update Q-table

    def get_optimal_policy(self):
        """Extracts the optimal policy after training."""
        return np.argmax(self.q_table, axis=1)

### Function to train the model using Q-learning for Frozen Lake

In [7]:
# ============================================================================================
# Function to train the model using Q-learning for Frozen Lake
# ============================================================================================
def run_tabular_q_frozen(env, agent, num_episodes=10, max_eps_len = 100, convergence_threshold=1e-4,
                         epsilon_start = 1, epsilon_decay = 0.995, epsilon_end = 0.01, seed=42):
    reward_curve = [] # this will store the moving avg of rewards
    moving_window = deque(maxlen=100)
    epsilon = epsilon_start
    prev_q_table = np.copy(agent.q_table)  # Store old Q-table

    for episode in tqdm(range(num_episodes)):
        state,_ = env.reset(seed=seed)
        # print(f"\nIn episode {episode}, After reset initial state = {state} and epsilon = {epsilon}")
        curr_reward = 0
        flag = False

        for _ in range(max_eps_len):
            action = agent.choose_action(state, epsilon)
            next_state, reward, done, _, _ = env.step(action)
            agent.update_q_value(state, action, reward, next_state)
            state = next_state
            curr_reward += reward

            # # Compute max Q-value change
            # q_change = np.max(np.abs(agent.q_table - prev_q_table))
            # prev_q_table = np.copy(agent.q_table)

            # # Check Q-value convergence
            # if q_change < convergence_threshold:
            #     print(f"Q-values converged at Episode {episode+1} with max Q-change: {q_change}")
            #     flag = True
            #     break

            if done:
                break
        # end while inside an episode
        
        # Epsilon decay performed at the end of each episode
        epsilon *= epsilon_decay
        epsilon = max(epsilon, epsilon_end)

        # Appending the smoothened reward
        moving_window.append(curr_reward)
        reward_curve.append(np.mean(moving_window))

        if episode % 1000 == 0:
            print(f"Tabular Q: Episode {episode}: epsilon = {epsilon}, avg reward = {np.mean(moving_window)}")
        # end if

        # if flag:
        #     break
    # end for num_episode

    return agent.q_table, agent.get_optimal_policy(), reward_curve

### Running the Tabular Q learning

In [8]:
# env = make_env(env_name=env_name, env_dim=env_dim, seed = seed, stochastic=stochastic)

# Setting seeds
np.random.seed(seed)
random.seed(seed)

if check_env_details:
    # Extract the environment description (grid layout)
    lake_grid = env.unwrapped.desc  # Gets the grid representation

    # Print state-to-symbol mapping
    print("Frozen Lake Grid Layout:")
    for row in lake_grid:
        print(" ".join(row.astype(str)))

    goal_state = None
    rows, cols = lake_grid.shape
    for i in range(rows):
        for j in range(cols):
            if lake_grid[i, j] == b'G':  # 'G' is stored as a byte-string
                goal_state = i * cols + j  # Convert (row, col) to state number
                break
        # end for j
    # end for i
    print(f"Goal State: {goal_state}")
# end if check_env

state_dim = env.observation_space.n
action_dim = env.action_space.n
print("State space: ", env.observation_space.n)
print("Action space: ", env.action_space.n)

learner = QLearningAgent(num_states=state_dim, num_actions=action_dim, gamma=gamma
                            , epsilon=epsilon_start, alpha=alpha) # Creating the learning Agent

final_q_table, final_policy, reward_curve = run_tabular_q_frozen(
                env, learner, num_episodes=num_episodes_q_table, max_eps_len=max_eps_len, convergence_threshold=convergence_threshold,
                epsilon_start = epsilon_start, epsilon_decay = epsilon_decay_q_table, epsilon_end = epsilon_end, seed=seed)

Val_f = np.max(final_q_table, axis=1)

state = 0
# Define action map
action_map = {
0: "Left",
1: "Down",
2: "Right",
3: "Up"
}
print("State: Type -    V(s),    action taken")
lake_grid = env.unwrapped.desc  # Gets the grid representation
for row in lake_grid:
    for cell in row:
        print(f"     {state}:   {cell.decode('utf-8')} - {Val_f[state]:.2f}, {final_policy[state]}-->{action_map[final_policy[state]]}")  # Convert byte to string
        state += 1
# assert False, "c1"

# Print the final table and policy
print("Final Q function: ", final_q_table)
# print("Final Policy: ", final_policy)
# print("Final Value function: ", Val_f)

# # Plot heatmap of the Value function
# plt.figure(figsize=(5,5))
# plt.imshow(Val_f.reshape(4,4), cmap="coolwarm", interpolation="nearest")
# for i in range(4):
#     for j in range(4):
#         plt.text(j, i, f"{Val_f[i*4+j]:.2f}", ha='center', va='center', color='black')


# Plot the reward curve



# Save the current Q-function
save_model = f"{runs_folder_path}/Q_table_{filename}.npy"
np.save(save_model, final_q_table)

Frozen Lake Grid Layout:
S F H F
F H F F
F F F H
F H F G
Goal State: 15
State space:  16
Action space:  4


  2%|▏         | 1697/100000 [00:00<00:05, 16968.84it/s]

Tabular Q: Episode 0: epsilon = 0.99995, avg reward = 0.0
Tabular Q: Episode 1000: epsilon = 0.9511806740132733, avg reward = 0.0
Tabular Q: Episode 2000: epsilon = 0.904789914112052, avg reward = 0.03
Tabular Q: Episode 3000: epsilon = 0.8606617134311852, avg reward = 0.02


  5%|▌         | 5071/100000 [00:00<00:05, 16529.47it/s]

Tabular Q: Episode 4000: epsilon = 0.8186857229650423, avg reward = 0.04
Tabular Q: Episode 5000: epsilon = 0.7787569756237134, avg reward = 0.04


...

Tabular Q: Episode 96000: epsilon = 0.01, avg reward = 0.98
Tabular Q: Episode 97000: epsilon = 0.01, avg reward = 0.99


 98%|█████████▊| 98265/100000 [00:05<00:00, 17723.00it/s]

Tabular Q: Episode 98000: epsilon = 0.01, avg reward = 0.97
Tabular Q: Episode 99000: epsilon = 0.01, avg reward = 0.98


100%|██████████| 100000/100000 [00:05<00:00, 17153.29it/s]

State: Type -    V(s),    action taken
     0:   S - 0.95, 1-->Down
     1:   F - 0.94, 0-->Left
     2:   H - 0.00, 0-->Left
     3:   F - 0.93, 1-->Down
     4:   F - 0.96, 1-->Down
     5:   H - 0.00, 0-->Left
     6:   F - 0.98, 1-->Down
     7:   F - 0.97, 0-->Left
     8:   F - 0.97, 2-->Right
     9:   F - 0.98, 2-->Right
     10:   F - 0.99, 1-->Down
     11:   H - 0.00, 0-->Left
     12:   F - 0.96, 3-->Up
     13:   H - 0.00, 0-->Left
     14:   F - 1.00, 2-->Right
     15:   G - 0.00, 0-->Left
Final Q function:  [[0.94148015 0.95099005 0.93206535 0.94148015]
 [0.94148015 0.         0.         0.93206535]
 [0.         0.         0.         0.        ]
 [0.         0.92984196 0.49090644 0.23199305]
 [0.95099005 0.96059601 0.         0.94148015]
 [0.         0.         0.         0.        ]
 [0.         0.9801     0.96059601 0.        ]
 [0.970299   0.         0.94446586 0.82700786]
 [0.96059601 0.95099005 0.970299   0.95099005]
 [0.96059601 0.         0.9801     0.        ]
 

# Collecting Trajectories
Here we may need to ensure that the trajectories sufficiently explore each state.\
Thus, we may want to start from specific states when reseting the environment during trajectory collection.

In [9]:
def choose_action(q_table, state, epsilon):
    """Epsilon-greedy action selection."""
    if random.uniform(0, 1) < epsilon:
        return random.randint(0, num_actions - 1)  # Explore
    else:
        return np.argmax(q_table[state, :])  # Exploit

In [10]:
q_table = np.load(f"{runs_folder_path}/Q_table_{filename}.npy")
# env = make_env(env_name=env_name, env_dim=env_dim, seed = seed, stochastic=stochastic)

traj = []
tuple_traj = False
for e in tqdm(range(num_episodes_trajectories)):
    # print(e)
    if modified=="perfect":
        _, _ = env.reset()
        epsilon = 0.01
        state = int(e%16)
    # end if perfect

    elif modified=="random":
        epsilon = 1
        if random.uniform(0, 1) < 0.1:
            if random.uniform(0, 1) < 0.5:
                state = 3
            elif random.uniform(0, 1) < 0.65:
                state = 6
            else:
                state = 7
            # end if choosing state
        else:
            state, _ = env.reset(seed=seed)
    # end if random trajectories
    
    else:
        # continue    
        if e<0.1*num_episodes_trajectories:
          epsilon = 1
        elif e<0.2*num_episodes_trajectories:
          epsilon = 0.8
        elif e<0.3*num_episodes_trajectories:
          epsilon = 0.6
        elif e<0.4*num_episodes_trajectories:
          epsilon = 0.4
        elif e<0.5*num_episodes_trajectories:
          epsilon = 0.2
        elif e<0.6*num_episodes_trajectories:
          epsilon = 0.1
        elif e<0.7*num_episodes_trajectories:
          epsilon = 0.05
        else:
          epsilon = 0.01
        # end epsilon if
        if random.uniform(0, 1) < 0.1:
            if random.uniform(0, 1) < 0.5:
                state = 3
            elif random.uniform(0, 1) < 0.65:
                state = 6
            else:
                state = 7
            # end if choosing state
        else:
            state, _ = env.reset(seed=seed)
    
    if not tuple_traj:
       traj.append('s_'+str(state))
    '''
    Action Stochasticity (is_slippery=True): The seed affects how the agent slips 
    (randomly moves instead of following the chosen action).
    Random Hole Placement (if map is generated dynamically): If the map has random 
    elements, different seeds can affect the placement of H (holes).
    '''
    curr_reward = 0
    rep_count = 0
    for t in range(max_eps_len):
      action = choose_action(q_table, state, epsilon) #To be implemented
      n_state,reward,done,_,_ = env.step(action)
      
      # We store the current tuple
      if tuple_traj:
        traj.append((state, action, reward, n_state, done))
      else:
        temp = 's_'+str(n_state)
        # traj.append('s_'+str(n_state))
        traj.append(temp)

      state = n_state
      curr_reward += reward
      if done:
        if rep_count>=5: # This forces repetitions to occur when done becomes True thereby repeating ending states
            if not tuple_traj:
                if temp=='s_15':
                    traj.append('s_'+str(16))
                else:
                    traj.append('s_'+str(17))
            break
        else:
            if not tuple_traj:
                traj.append(temp)
        rep_count+=1
    # end for
# end for
if tuple_traj:
    save_file_name = f"{runs_folder_path}/modified_tuple_trajectories_{filename}.npy"
    np.save(save_file_name, traj)
else:
    save_file_name = f"{runs_folder_path}/modified_{modified}_trajectories_{filename}.npy"
    np.save(save_file_name, traj)
print(f"Trajectories Saved in {save_file_name}!")

# assert False, "No w2v business here"

100%|██████████| 10000/10000 [00:00<00:00, 18772.43it/s]

Trajectories Saved in mdp/runs_frozen/modified_perfect_trajectories_FrozenLake-v1_map_size_4_stochastic_False_seed_42.npy!


# Performing w2v

In [11]:
traj_file = f"{runs_folder_path}/modified_{modified}_trajectories_{filename}.npy"
print("The trajectories being used are: ", traj_file)

text = np.load(traj_file)
np.random.seed(seed=seed)
torch.manual_seed(seed)

The trajectories being used are:  mdp/runs_frozen/modified_perfect_trajectories_FrozenLake-v1_map_size_4_stochastic_False_seed_42.npy


### Now we define the functions for word2vec algorithm

In [12]:
def build_vocab(text): # Again we already have a vocabulary hence don't need to use this function directly
    word_counts = Counter(text)
    vocab = {word: i for i, word in enumerate(word_counts.keys())}
    reverse_vocab = {i: word for word, i in vocab.items()}
    return vocab, reverse_vocab, word_counts

def generate_skipgram_pairs(text, window_size=2): # This function gives the word and context pairs.
    # words = tokenize_text(text)
    words = text
    pairs = []
    for i, target_word in enumerate(words):
        window_start = max(i - window_size, 0)
        window_end = min(i + window_size + 1, len(words))
        for j in range(window_start, window_end):
            if i != j:
                pairs.append((words[i], words[j]))
    return pairs

# vocab,_,wcounts = build_vocab(text)
# print("vocab: ", vocab)
# print("word counts: ", wcounts)
# assert False, "Checking the word counts!"

### Word2vec class definition

In [13]:
# ============================================================================================
# Classes: Word2vec Dataset creator and SkipGram model
# ============================================================================================

class Word2VecDataset(Dataset):
    def __init__(self, text, vocab, window_size=2):
        self.vocab = vocab
        self.data = generate_skipgram_pairs(text, window_size)
        self.vocab_size = len(vocab) # why is this needed here?

    def __len__(self): # what does this function do?
        return len(self.data)

    def __getitem__(self, idx):
        target, context = self.data[idx]
        ''' Doesn't the above idx reflect the idx numbered pair instead in pairs instead of the pairs
        corresponding to the word at idx? '''
        target_idx = torch.tensor(self.vocab[target], dtype=torch.long)
        context_idx = torch.tensor(self.vocab[context], dtype=torch.long)
        # target_idx = self.vocab[target]
        # context_idx = self.vocab[context]

        return target_idx, context_idx
    

    # ============================================================================================
# Class: SkipGram using softmax over entire vocabulary
# ============================================================================================

class SkipGram(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGram, self).__init__()
        self.embedding_dim = embedding_dim

        # Input word embedding
        self.in_embedding = nn.Embedding(vocab_size, embedding_dim)
        # Output word embedding (used for context words)
        self.out_embedding = nn.Embedding(vocab_size, embedding_dim)

        # Initialize weights (better stability)
        init_range = 0.5 / embedding_dim
        self.in_embedding.weight.data.uniform_(-init_range, init_range)
        self.out_embedding.weight.data.uniform_(-init_range, init_range)

    def forward(self, center_word_idx):
        """Compute word embeddings and softmax probabilities for the context words."""
        center_embed = F.relu(self.in_embedding(center_word_idx))  # Shape: (batch_size, embedding_dim)
        scores = torch.matmul(center_embed, self.out_embedding.weight.T)  # Compute dot product
        y_pred = torch.softmax(scores, dim=1)  # Apply softmax over output vocab
        return y_pred

    def get_word_vector(self, word_idx):
        """Return the learned embedding vector for a given word index."""
        return self.in_embedding(word_idx).detach().cpu().numpy()

### Training function

In [14]:
# ============================================================================================
# Function: to train the w2v skipgram model
# ============================================================================================

def train_skipgram(model, data_loader, epochs=6, lr=0.01, device = device):
    """Train the SkipGram model using Adam optimizer."""
    criterion = nn.CrossEntropyLoss()  # Cross-entropy for multi-class classification
    optimizer = optim.Adam(model.parameters(), lr)

    for epoch in tqdm(range(epochs)):
        total_loss = 0
        for center_word_idx, context_word_idx in data_loader:
            center_word_idx = center_word_idx.to(device)
            context_word_idx = context_word_idx.to(device)

            optimizer.zero_grad()
            y_pred = model(center_word_idx)  # Forward pass
            loss = criterion(y_pred, context_word_idx)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights

            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

    return model

### Main function for w2v

In [15]:
# ============================================================================================
#  Main function for calling the w2v agent
# ============================================================================================
# First we create the dataset and dataloader
vocab, _, _ = build_vocab(text)
dataset = Word2VecDataset(text, vocab, window_size)
dataloader = DataLoader(dataset, batch_size, shuffle=True)
model = SkipGram(vocab_size=len(vocab), embedding_dim=embed_dim).to(device)
model = train_skipgram(model, dataloader, epochs = w2v_epochs, lr = w2v_lr)

# ============================================================================================
#  Saving the w2v generated vector embeddings as a dictionary
# ============================================================================================
word_embeddings = {}
for word in vocab:
    word_idx = torch.tensor([vocab[word]], dtype=torch.long).to(device)
    updated_embedding = model.get_word_vector(word_idx)
    # print(f"Updated embedding for '{word}': {updated_embedding}")
    # Store the embedding in the dictionary
    word_embeddings[word] = updated_embedding.flatten()  # Flatten to 1D array

# Saving the w2v model
save_w2v_file = f"{runs_folder_path}/modified_{modified}_w2v_embed_dim_{embed_dim}_{filename}_epochs_{w2v_epochs}.npy"
np.save(save_w2v_file, word_embeddings)
print("W2v vetors stored in: ", save_w2v_file)

  2%|▏         | 1/50 [00:15<12:35, 15.42s/it]

Epoch 1, Loss: 110887.2898


  4%|▍         | 2/50 [00:32<12:58, 16.22s/it]

Epoch 2, Loss: 110893.1169


  6%|▌         | 3/50 [00:49<12:55, 16.49s/it]

...

Epoch 48, Loss: 113417.2645


 98%|█████████▊| 49/50 [12:45<00:14, 14.95s/it]

Epoch 49, Loss: 113417.2709


100%|██████████| 50/50 [13:00<00:00, 15.61s/it]

Epoch 50, Loss: 113417.2790
W2v vetors stored in:  mdp/runs_frozen/modified_perfect_w2v_embed_dim_32_FrozenLake-v1_map_size_4_stochastic_False_seed_42_epochs_50.npy


### Similarity check

In [16]:
# ============================================================================================
# Similarity checking and visualizing 
# ============================================================================================
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity

# Load saved embeddings
word_embeddings = np.load(f"{runs_folder_path}/modified_{modified}_w2v_embed_dim_{embed_dim}_{filename}_epochs_{w2v_epochs}.npy", 
                          allow_pickle=True).item()

# Convert to a NumPy array for fast computation
words = list(word_embeddings.keys())
vectors = np.array(list(word_embeddings.values()))

# Compute the cosine similarity matrix
cosine_sim_matrix = cosine_similarity(vectors)

# Function to find top-N similar words
def find_similar_words(target_word, top_n=5):
    if target_word not in word_embeddings:
        print(f"Word '{target_word}' not found in vocabulary.")
        return []

    # Get index of target word
    target_idx = words.index(target_word)

    # Get similarity scores for the target word
    similarity_scores = cosine_sim_matrix[target_idx]

    # Get top-N most similar words (excluding itself)
    similar_indices = np.argsort(similarity_scores)[::-1][1:top_n+1]  # Sort in descending order

    # Return words with their similarity scores
    return [(words[i], similarity_scores[i]) for i in similar_indices]

# Example usage
for target_word in word_embeddings:
    top_similar_words = find_similar_words(target_word, top_n=5)

    print(f"Top 5 words similar to '{target_word}':")
    for word, score in top_similar_words:
        print(f"{word}: {score:.4f}")

Top 5 words similar to 's_0':
s_7: 0.8252
s_9: 0.5571
s_6: 0.5460
s_8: 0.5050
s_12: 0.4732
Top 5 words similar to 's_4':
s_10: 0.5551
s_1: 0.4990
s_0: 0.4210
s_7: 0.4177
s_5: 0.4088
Top 5 words similar to 's_8':
s_11: 0.6781
s_10: 0.5752
s_5: 0.5568
s_3: 0.5273
s_0: 0.5050
Top 5 words similar to 's_9':
s_0: 0.5571
s_5: 0.5321
s_7: 0.4793
s_6: 0.4463
s_1: 0.4430
Top 5 words similar to 's_10':
s_8: 0.5752
s_5: 0.5638
s_4: 0.5551
s_2: 0.5457
s_3: 0.5308
Top 5 words similar to 's_14':
s_16: 0.3096
s_15: 0.0496
s_13: -0.0069
s_17: -0.0252
s_9: -0.0977
Top 5 words similar to 's_15':
s_16: 0.3565
s_14: 0.0496
s_13: -0.0071
s_17: -0.0441
s_8: -0.0796
Top 5 words similar to 's_16':
s_15: 0.3565
s_14: 0.3096
s_17: 0.2524
s_13: -0.0040
s_9: -0.1480
Top 5 words similar to 's_1':
s_5: 0.5043
s_4: 0.4990
s_2: 0.4935
s_9: 0.4430
s_12: 0.4274
Top 5 words similar to 's_11':
s_8: 0.6781
s_2: 0.6654
s_3: 0.6033
s_5: 0.5503
s_7: 0.5113
Top 5 words similar to 's_17':
s_16: 0.2524
s_6: 0.0410
s_8: 0.0119
s_

# Now we need to ensure that the same states are sampled to compare the performance of w2v imbued DQN and Vanilla DQN. 
Thus we reset np, random and torch.manual_seed before starting both the following cells

# DQN common set up

### Defining all classes
We define the following classes:
1. Experience Replay
2. DQNAgent 

### Training using w2v

### Vanilla DQN (without w2v)